In [ ]:
    async def get_enhanced_ect(
        self,
        ticker: str,
        year: int,
        quarter: str,
        as_structure: bool = False,
        response_as_html: bool = False,
        html_prompt: str = "",
    ) -> Union[List[Any], Dict[str, Any]]:
        """Get enhanced ECT analysis including customer and supplier insights"""
        
        # Initialize RelationDB
        rel_db = RelationDB(year=year, quarter=quarter, use_cache=True)
        
        # Get company info
        id_bb_company = rel_db.ticker_to_bbg_cid(ticker)
        if not id_bb_company:
            return [f"Could not find company ID for ticker {ticker}"]
            
        company_name = rel_db.d["bbg cid to name map"].get(id_bb_company)
        if not company_name:
            return [f"Could not find company name for ID {id_bb_company}"]

        # Get relationships data
        customers = rel_db.get_customers_by_ticker(ticker)
        suppliers = rel_db.get_suppliers_by_ticker(ticker)
        bbg_cid_to_name_map = rel_db.d["bbg cid to name map"]

        # Get ECT data for company and its relationships
        all_partners = [
            {
                "id_bb_company": x,
                "refinitiv_id": rel_db.ticker_to_refinitive_id(rel_db.bbg_cid_to_ticker(x))
            }
            for x in [id_bb_company] + list(customers.keys()) + list(suppliers.keys())
            if rel_db.bbg_cid_to_ticker(x)  # Only include if we can get the ticker
        ]

        # Get ECT responses
        ect_responses = await self.get_ect_for_companies(all_partners, period="4m")

        # Process prompts and sources
        ect_prompts = []
        customer_list = []
        supplier_list = []
        company_sources = []
        customer_sources = []
        supplier_sources = []

        # Process main company ECT
        if company_ect := ect_responses.get(id_bb_company):
            if sources := company_ect.get("sources"):
                company_sources = [{
                    "name": company_name,
                    "tr_file_name": sources.get("TR_File_Name", "")
                }]
                ect_prompts = self.generate_ect_prompts(
                    company_ect["ect"],
                    id_bb_company,
                    html_prompt
                )

        # Process initial responses
        prompt_list = ect_prompts + customer_list + supplier_list
        responses = await self.execute_prompts(prompt_list)

        ect_output = []
        customer_output = []
        supplier_output = []

        for response in responses:
            if response["type"] == "ECT" and response["confidence"] > self.config.llm.confidence_threshold:
                ect_output.append(response["result"])
            elif response["type"] == "Customer":
                customer_output.append(response)
            elif response["type"] == "Supplier":
                supplier_output.append(response)

        ect_output = "\n\n".join(ect_output) if ect_output else ""

        # Prepare data for summary prompts
        customer_names = ", ".join(bbg_cid_to_name_map[cid] for cid in customers.keys())
        supplier_names = ", ".join(bbg_cid_to_name_map[cid] for cid in suppliers.keys())
        
        customers_cons_data = {k: v for k, v in rel_db.d["cons data final struct"].items() 
                             if k in customers}
        suppliers_cons_data = {k: v for k, v in rel_db.d["cons data final struct"].items() 
                             if k in suppliers}

        # Generate final summaries
        summary_prompts = []
        
        # Customer summary
        if customers:
            customer_prompt = self._generate_customer_summary_prompt(
                company_name=company_name,
                customer_names=customer_names,
                customers_cons_data=str(customers_cons_data),
                customer_responses=str(customer_output),
                bbg_cid_to_name_map=bbg_cid_to_name_map,
                id_bb_company=id_bb_company,
                html_prompt=html_prompt
            )
            summary_prompts.append({
                "type": "Customer",
                "prompt": customer_prompt,
                "company": id_bb_company
            })

        # Supplier summary
        if suppliers:
            supplier_prompt = self._generate_supplier_summary_prompt(
                company_name=company_name,
                supplier_names=supplier_names,
                suppliers_cons_data=str(suppliers_cons_data),
                supplier_responses=str(supplier_output),
                bbg_cid_to_name_map=bbg_cid_to_name_map,
                id_bb_company=id_bb_company,
                html_prompt=html_prompt
            )
            summary_prompts.append({
                "type": "Supplier",
                "prompt": supplier_prompt,
                "company": id_bb_company
            })

        # Execute summary prompts
        summary_responses = []
        if summary_prompts:
            summary_responses = await self.execute_prompts(
                summary_prompts,
                title="Supply Chain Final Summary"
            )

        # Convert to HTML if requested
        if response_as_html:
            ect_output = self._convert_to_html(ect_output)
            for resp in summary_responses:
                resp["result"] = self._convert_to_html(resp["result"])

        # Format output
        if as_structure:
            out = [
                ect_output,
                summary_responses[0]["result"] if summary_responses else "",
                summary_responses[1]["result"] if len(summary_responses) > 1 else "",
                company_sources,
                customer_sources,
                supplier_sources
            ]
        else:
            out = [
                ect_output,
                summary_responses[0]["result"] if summary_responses else "",
                summary_responses[1]["result"] if len(summary_responses) > 1 else ""
            ]

        return out

In [ ]:
def _generate_customer_summary_prompt(
    self,
    company_name: str,
    customer_names: str,
    customers_cons_data: str,
    customer_responses: str,
    bbg_cid_to_name_map: Dict[str, str],
    id_bb_company: str,
    html_prompt: str = "",
) -> str:
    return f"""Following is the top ranked customers for {company_name}: lower rank is better.

Following is the consensus quarterly revenue growth over next 4 quarters for the below companies: {customers_cons_data}
Following is the summary of earnings transcripts for customers of companies \n%s

Using the above information on top customers and earnings transcript summary of the customers' growth prospects, what do you think of the future revenue prospects of these companies?
Important : Provide the answer as a table with the heading '{bbg_cid_to_name_map[id_bb_company]} top customers' and having only company names with valid earnings transcript summary.
and sorted by customer ranking as rows and 7 columns - customer ranking , 'Customer Name', 'Sector' , 'Qtrly Growth Next 4Q', 'Key growth driver' , 'Revenue growth prospects'.
'Key growth driver' should specify max 3 words on the key growth driver for the company.
'Revenue growth prospects' columns should classify the customers as 'fast-paced growth', 'slow-paced growth' or 'flat /negative growth'
using only the information from earnings transcript summary. 'Period' should be the quarter and year for the earnings transcript summary.
Add brief comments on the logic and reasoning for each company after the table in the response.
Altogether remove companies with no earnings transcript information from the response.
Add a footnote that Qtrly Growth Next 4Q and customer relationship is from Factset.""".strip()

def _generate_supplier_summary_prompt(
    self,
    company_name: str,
    supplier_names: str,
    suppliers_cons_data: str,
    supplier_responses: str,
    bbg_cid_to_name_map: Dict[str, str],
    id_bb_company: str,
    html_prompt: str = "",
) -> str:
    return f"""Following is the top ranked suppliers for {company_name}: lower rank is better.

Following is the consensus quarterly revenue growth over next 4 quarters for the below companies: {suppliers_cons_data}
Following is the summary of earnings transcripts for suppliers of companies \n%s

Using the above information on top suppliers and earnings transcript summary of the suppliers' pricing power, what do you think of the pricing power of these companies?
Important : Provide the answer as a table with the heading '{bbg_cid_to_name_map[id_bb_company]} top suppliers' and having only company names with valid earnings transcript summary and
sorted by supplier ranking as rows and 8 columns - 
'Supplier ranking', 'Supplier Name', 'Sector', 'Qtrly Growth Next 4Q', 'Pricing Power', 'Inventory backlog', 'Equipment Expenses', 'Product Strategy' and 'Period'.
'Product Strategy' should be a 2 to 3 word summary of information from earnings transcript.
'Pricing Power' column should classify the suppliers as 'high', 'medium', 'low' or 'unknown' using the information from earnings transcript summary.
'Inventory backlog' column should classify the suppliers as 'high', 'medium', 'low' or 'unknown' using the information from earnings transcript summary.
'Equipment Expenses' column should classify the suppliers as 'increase', 'decrease', or 'unknown' using the information from earnings transcript summary.
'Qtrly Growth Next 4Q' should show the average quarterly revenue growth rate for the next 4 quarters. 'Period' should be the quarter and year for the earnings transcript summary.
Add brief comments on the logic and reasoning for each company after the table in the response.
Altogether remove companies with no earnings transcript information from the response.
Add a footnote that Qtrly Growth Next 4Q and supplier relationship is from Factset.""".strip()